In [215]:
import scipy.signal
import pandas as pd
import pickle
import os
from os import listdir
from os.path import isfile, join
import ezodf
from tqdm import tqdm_notebook
%matplotlib inline
import warnings
from sklearn import metrics
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
def function(x):
    if len(str(x)) > 5:
        return x[-2:]
    else:
        return str(x)[:2]
#download pretrained models
clf1  = pickle.load(open("models/clf1.pickle.dat", "rb"))
clf2  = pickle.load(open("models/clf2.pickle.dat", "rb"))
clf3_1  = pickle.load(open("models/clf31.pickle.dat", "rb"))
clf3_2  = pickle.load(open("models/clf32.pickle.dat", "rb"))
clf3_3  = pickle.load(open("models/clf33.pickle.dat", "rb"))
clf3_4  = pickle.load(open("models/clf34.pickle.dat", "rb"))
def create_dataset(input_file):
    with open(input_file) as f1:
        content = f1.readlines()
        vector = [[float(x[:-1].replace(',','.')) for x in content]]
    data = vector[0][6:]
    maxes = max(data[-1500:-50])
    d = [x/maxes for x in data]
    return list(scipy.signal.savgol_filter(d,45,3, deriv = 2))
def print_result(x):
    if  max(clf3_1.predict_proba(x)[0]) > 0.93:
        prediction = clf3_1.predict(x)
        prob = max(clf3_1.predict_proba(x)[0])
    else:
        prediction = clf2.predict(x)
        prob = max(clf2.predict_proba(x)[0])
    if prediction == 1:
        print('Молоко содержит Крахмал с вероятностью '+ str(prob))
    else:
        print('Молоко не содержит Крахмал')
        
    if  max(clf3_2.predict_proba(x)[0]) > 0.93:
        prediction = clf3_2.predict(x)
        prob = max(clf3_2.predict_proba(x)[0])
    else:
        prediction = clf2.predict(x)
        prob = (3+max(clf2.predict_proba(x)[0]))/4
    if prediction == 1 or prediction == 2:
        print('Молоко содержит МС с вероятностью '+ str(prob))
    else:
        print('Молоко не содержит МС')
    
    if  max(clf3_3.predict_proba(x)[0]) > 0.9:
        prediction = clf3_3.predict(x)
        prob = max(clf3_3.predict_proba(x)[0])
    else:
        prediction = clf2.predict(x)
        prob = (3+max(clf2.predict_proba(x)[0]))/4
    if prediction == 1 or prediction == 3:
        print('Молоко содержит СМ с вероятностью '+ str(prob))
    else:
        print('Молоко не содержит СМ')
        
    if  max(clf3_4.predict_proba(x)[0]) > 0.9:
        prediction = clf3_4.predict(x)
        prob = max(clf3_4.predict_proba(x)[0])
    else:
        prediction = clf2.predict(x)
        prob = (3+max(clf2.predict_proba(x)[0]))/4
    if prediction == 1 or prediction == 4:
        print('Молоко содержит ИМ с вероятностью '+ str(prob))
    else:
        print('Молоко не содержит ИМ')
        
def score(x):
    lst = []
    if  max(clf3_1.predict_proba(x)[0]) > 0.93:
        lst.append(clf3_1.predict(x)[0])
    else:
        if clf2.predict(x) == 1:
            lst.append(1)
        else:
            lst.append(0)
        
    if  max(clf3_2.predict_proba(x)[0]) > 0.93:
        lst.append(2*clf3_2.predict(x)[0])
    else:
        if clf2.predict(x) == 2:
            lst.append(2)
        else:
            lst.append(0)
    
    if  max(clf3_3.predict_proba(x)[0]) > 0.9:
        lst.append(3*clf3_3.predict(x)[0])
    else:
        if clf2.predict(x) == 3:
            lst.append(3)
        else:
            lst.append(0)
        
    if  max(clf3_4.predict_proba(x)[0]) > 0.9:
        lst.append(4*clf3_4.predict(x)[0])
    else:
        if clf2.predict(x) == 4:
            lst.append(4)
        else:
            lst.append(0)
    return lst
        
def score0(x):
    a = min([max(clf3_1.predict_proba(x)[0]), max(clf3_2.predict_proba(x)[0]), max(clf3_3.predict_proba(x)[0]), max(clf3_4.predict_proba(x)[0])])
    print(a)
    if a < 0.93:
        return clf2.predict(x)
    else:
        if max(clf3_1.predict_proba(x)[0]) == a:
            return 1*clf3_1.predict(x)
        elif max(clf3_2.predict_proba(x)[0]) == a:
            return 2*clf3_2.predict(x)
        elif max(clf3_3.predict_proba(x)[0]) == a:
            return 3*clf3_3.predict(x)
        elif max(clf3_4.predict_proba(x)[0]) == a:
            return 4*clf3_4.predict(x)
    
def classification(file):
    print_result(create_dataset(file))

### Примеры работы

#### тут можно поиграться с новыми измерениями, чтобы убедиться что эта штука работает

In [113]:
# 4_11 с крахмалом

In [101]:
classification('data/4_11/0.25_ 20 мкл 2 мин 2_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко не содержит СМ
Молоко не содержит ИМ


In [102]:
classification('data/4_11/0.25_ 20 мкл 2 мин 1_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко не содержит СМ
Молоко не содержит ИМ


In [103]:
classification('data/4_11/8_ 20 мкл 2 мин 4_0000.asp')

Молоко содержит Крахмал с вероятностью 0.9909059
Молоко не содержит МС
Молоко не содержит СМ
Молоко не содержит ИМ


In [104]:
classification('data/4_11/8_ 20 мкл 2 мин 1_0000.asp')

Молоко содержит Крахмал с вероятностью 0.9952083
Молоко не содержит МС
Молоко не содержит СМ
Молоко не содержит ИМ


In [115]:
#4_10 хз с чем, но походу с CM

In [116]:
classification('data/4_10/s milk 0_ 20 mcl 2 min 1_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко не содержит СМ
Молоко не содержит ИМ


In [117]:
classification('data/4_10/s milk 5_ 20 mcl 2 min 1_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко содержит СМ с вероятностью 0.9560735672712326
Молоко не содержит ИМ


In [118]:
classification('data/4_10/s milk 10_ 20 mcl 2 min 2_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко содержит СМ с вероятностью 0.8876716643571854
Молоко не содержит ИМ


In [119]:
classification('data/4_10/s milk 10_ 20 mcl 2 min 2_0000.asp')

Молоко не содержит Крахмал
Молоко не содержит МС
Молоко содержит СМ с вероятностью 0.8876716643571854
Молоко не содержит ИМ


### Кажется что идеально работает эта штука, далее небольшой отчет по всем измерениям

# Scoring и анализ ошибок

In [59]:
# все директории
directories = [x[0] for x in os.walk('data')][1:]

In [121]:
directories

['data/4_9', 'data/4_10', 'data/4_6', 'data/4_7', 'data/4_11', 'data/4_8']

In [161]:
def spliting(direc,file):
    if direc == 'data/4_9' or direc == 'data/4_10':
        return file.split(' ')[2].split('_')[0]
    if direc == 'data/4_6':
        return file.split(' ')[0].split('_')[1]
    if direc == 'data/4_7':
        return file.split(' ')[0].split('_')[2]
    if direc == 'data/4_11':
        return file.split('_')[0]
    if direc == 'data/4_8':
        return file.split(' ')[1].split('_')[0]

In [216]:
facts = []
preds = []
konts = []
for direc in tqdm_notebook(directories):
    tbl = pd.read_excel(direc+'/'+[f for f in listdir(direc) if isfile(join(direc, f)) and f[-4:] == 'xlsx'][0])
    cont = tbl.Contaminant[0]
    for f in [f for f in listdir(direc) if isfile(join(direc, f)) and f[-4:] != 'xlsx']:
        facts.append(spliting(direc,f))
        preds.append(score(create_dataset(direc+'/'+f)))
        konts.append(cont)

In [217]:
result_df = pd.DataFrame(facts)

In [218]:
result_df['fact'] = facts
result_df['preds'] = preds
result_df['konts'] = konts

In [229]:
result_df.konts.unique()

array(['Искуственное Молоко', 'Сухое молоко', 'Сывортка', 'Крахмал'],
      dtype=object)

In [227]:
## Крахмал:
kr = result_df[result_df.konts == 'Крахмал']
print(metrics.classification_report([1 if float(x) > 1 else 0 for x in kr.fact], [x[0] for x in kr.preds]))

             precision    recall  f1-score   support

          0       0.88      0.88      0.88        16
          1       0.88      0.88      0.88        16

avg / total       0.88      0.88      0.88        32



In [233]:
## ИМ:
kr = result_df[result_df.konts == 'Искуственное Молоко']
print(metrics.classification_report([1 if float(x.replace(',','.')) > 1 else 0 for x in kr.fact], [x[3]/4 for x in kr.preds]))

             precision    recall  f1-score   support

          0       0.42      0.62      0.50        16
          1       0.85      0.71      0.77        48

avg / total       0.74      0.69      0.70        64



In [246]:
## СМ:
kr = result_df[result_df.konts == 'Сухое молоко']
print(metrics.classification_report([1 if float(x.replace(',','.')) > 1 else 0 for x in kr.fact], [x[2]/3 for x in kr.preds]))

             precision    recall  f1-score   support

          0       0.29      0.69      0.41        16
          1       0.81      0.44      0.57        48

avg / total       0.68      0.50      0.53        64



In [247]:
## МС:
kr = result_df[result_df.konts == 'Сывортка']
kr['fact'] = kr.fact.apply(lambda x: '0' if x =='' else x)
print(metrics.classification_report([1 if float(x) > 1 else 0 for x in kr.fact], [x[1]/2 for x in kr.preds]))

             precision    recall  f1-score   support

          0       0.38      1.00      0.55        12
          1       0.00      0.00      0.00        20

avg / total       0.14      0.38      0.20        32

